## Assignment: Segmenting and clustering neighborhoods in Toronto

### I have used one single notebook for all three parts of the Assignment

For this assignment, we are required to explore and cluster the neighborhoods in Toronto.



Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 

# Part 1:

In [99]:
import requests
from bs4 import BeautifulSoup
import csv
import json
import xml
import pandas as pd

In [100]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [101]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(website_url,'lxml')

print(soup)


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":900271985,"wgRevisionId":900271985,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","S

In [102]:
neighborhood = soup.find('table', class_ = 'wikitable')
neighborhood_rows = neighborhood.find_all('tr')

### Extracting the columns 'Postcode', 'Borough', 'Neighbourhood' from the table

In [103]:
information = []
for row in neighborhood_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (the first and last items)
    information.append(info)
    
information[0:10] 

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned']]

### converting the table into a Pandas dataframe

In [78]:
neighbor_df = pd.DataFrame(information[1:], columns=information[0])
# where information[1:] contains each row of neighborhoods
# and columns = information[0] gives the column names


# Re-spell Neighbourhood as Neighborhood
neighbor_df = neighbor_df.rename(columns={neighbor_df.columns[2]: "Neighborhood" })


neighbor_df.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Ignore cells with a borough that is Not assigned.


In [79]:
neighbor_df = neighbor_df[neighbor_df.Borough != 'Not assigned']

neighbor_df.reset_index(drop=True, inplace=True)

neighbor_df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Grouping neighbourhoods with same postcode 

In [104]:
grouped = neighbor_df.groupby(['Postcode']) # group by Postcode


# combine the neighborhoods grouped by postcode and into a new df
neighborhood_grouped = grouped['Neighborhood'].apply(lambda x: x.sum()) 
# adds spaces and commas between neighborhoods
neighborhood_grouped = grouped['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
# matches a borough to each postcode
borough_grouped = grouped['Borough'].apply(lambda x: set(x).pop())
# turn borough_grouped and neighborhood_grouped into dataframes
borough = borough_grouped.to_frame()
neighborhood = neighborhood_grouped.to_frame()
#combine the dataframe borough and the dataframe neighborhood into one dataframe
grouped_final = borough.merge(neighborhood, on="Postcode")

grouped_final

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


### print the number of rows of your dataframe

In [80]:
grouped_final.shape

(103, 2)

# Part 2:

Get The geospatial data from the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [108]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')

geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [114]:
## Rename Postal code to Postcode
geospatial_data = geospatial_data.rename(columns={geospatial_data.columns[0]: "Postcode" }) 

In [115]:
full_table = grouped_final.merge(geospatial_data, on = 'Postcode')

full_table.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3:

### Explore and cluster the neighborhoods in Toronto.

In [116]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Find total number of boroughs and neighborhoods in Toronto

In [117]:
print('The dataframe has total {} boroughs and {} neighborhoods.'.format(
        len(full_table['Borough'].unique()),
        full_table['Neighborhood'].shape[0]
    )
)

The dataframe has total 11 boroughs and 103 neighborhoods.


In [118]:
# get the longitude and latitude of Toronto 

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [87]:
# create a map of Toronto
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor, neigh in zip(full_table['Latitude'], full_table['Longitude'], 
                                 full_table['Borough'], full_table['Neighborhood']):
    label = '{}, {}'.format(neigh, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
        
map_toronto

In [88]:
list_boroughs = full_table['Borough'].unique()
list_boroughs

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [90]:
def borough_loc(list_of_places):
    for place in list_of_places:
        address = (place + ", Ontario, Canada")
        geolocator = Nominatim(user_agent="TO_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('{''}, {}, {},'.format(place,latitude,longitude))

borough_loc(list_boroughs)

Scarborough, 43.773077, -79.257774,
North York, 43.7708175, -79.4132998,
East York, 43.6913391, -79.3278212,
East Toronto, 43.6247901, -79.3934918,
Central Toronto, 43.653963, -79.387207,
Downtown Toronto, 43.655115, -79.380219,
York, 44.0007518, -79.4372217,
West Toronto, 43.653963, -79.387207,
Queen's Park, 43.6599803, -79.3903686,
Mississauga, 43.590338, -79.645729,
Etobicoke, 43.67145915, -79.5524920661167,


In [91]:
import numpy as np

boroughs=[
'Scarborough', 43.773077, -79.257774,
'North York', 43.7708175, -79.4132998,
'East York', 43.6913391, -79.3278212,
'East Toronto', 43.6247901, -79.3934918,
'Central Toronto', 43.653963, -79.387207,
'Downtown Toronto', 43.655115, -79.380219,
'York', 44.0007518, -79.4372217,
'West Toronto', 43.653963, -79.387207,
"Queen's Park", 43.6599803, -79.3903686,
'Mississauga', 43.590338, -79.645729,
'Etobicoke', 43.67145915, -79.5524920661167]

boroughs_df = pd.DataFrame(np.array(boroughs).reshape(11,3), columns = ["Borough","Latitude","Longitude"])

boroughs_df

,Borough,Latitude,Longitude
0,Scarborough,43.773077,-79.257774
1,North York,43.7708175,-79.4132998
2,East York,43.6913391,-79.3278212
3,East Toronto,43.6247901,-79.3934918
4,Central Toronto,43.653963,-79.387207
5,Downtown Toronto,43.655115,-79.380219
6,York,44.0007518,-79.4372217
7,West Toronto,43.653963,-79.387207
8,Queen's Park,43.6599803,-79.3903686
9,Mississauga,43.590338,-79.645729


In [92]:
boroughs_df.dtypes

boroughs_df["Latitude"] = pd.to_numeric(boroughs_df["Latitude"])
boroughs_df["Longitude"] = pd.to_numeric(boroughs_df["Longitude"])


In [93]:
# create a map of Toronto using location of central toronto
map_toronto_boroughs = folium.Map(location = [43.653963, -79.387207], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor in zip(boroughs_df['Latitude'], boroughs_df['Longitude'], 
                                 boroughs_df['Borough']):
    label = '{}'.format(bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto_boroughs)
        
map_toronto_boroughs

In [94]:
full_table['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [95]:
# The geographical coordinates for Downtown Toronto
DT_lat = boroughs_df.iloc[5,1]
DT_long = boroughs_df.iloc[5,2]

# The dataframe that contains all the Downtown Toronto neighborhoods
DT_df = full_table[full_table['Borough'] == 'Downtown Toronto'].reset_index(drop = True)

# create map of Downtown Toronto neighborhoods using latitude and longitude values
map_DT = folium.Map(location=[DT_lat, DT_long], zoom_start=12.4)

# add markers to map
for lat, lng, label in zip(DT_df['Latitude'], DT_df['Longitude'], DT_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

In [96]:
# Define Foursquare Credentials and Versions

CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxxxxxxxxxxx' # your Foursquare Secret
VERSION = '20181102' # Foursquare API version

In [97]:
DT_df.loc[0, 'Neighborhood']

'Rosedale'

In [98]:
neighborhood_latitude = DT_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DT_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DT_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [48]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    lat, 
    lng, 
    VERSION, 
    radius, 
    LIMIT)


In [49]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2805976c0aa500305fe91b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4adcfd7cf964a5203e6321e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '4adcfd7cf964a5203e6321e3',
       'location': {'address': '200 Christie St',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Essex St',
        'distance': 205,
        'formattedAddress': ['200 Christie St (at Essex St)',
         'Toronto ON M6G 3B6',
         'Canada'],
        'labeled

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [52]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Fiesta Farms,Grocery Store,43.668471,-79.420485
1,Contra Cafe,Café,43.669107,-79.426105
2,Vinny’s Panini,Italian Restaurant,43.670679,-79.426148
3,Starbucks,Coffee Shop,43.671585,-79.421366
4,Universal Grill,Diner,43.670550,-79.426541
5,Actinolite,Restaurant,43.667858,-79.428054
6,Scout and Cash Caffe,Café,43.667360,-79.419938
7,Faema Caffe,Café,43.672003,-79.422209
8,Loblaws,Grocery Store,43.671807,-79.421102
9,Marlenes Just Babies,Baby Store,43.671824,-79.420499


In [53]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                lat, 
                lng, 
                VERSION, 
                radius, 
                LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
DT_venues = getNearbyVenues(names=DT_df['Neighborhood'],
                                   latitudes=DT_df['Latitude'],
                                   longitudes=DT_df['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [57]:
print(DT_venues.shape)
DT_venues.head()

(1294, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [58]:
DT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [59]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [60]:
DT_onehot.shape

(1294, 203)

In [61]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00,0.00,0.0,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011364,...,0.000000,0.00,0.00,0.0,0.00,0.011364,0.000000,0.000000,0.011364,0.000000
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.01,0.00,0.0,0.00,0.060000,0.000000,0.050000,0.010000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.011628,0.00,0.00,0.0,0.00,0.000000,0.011628,0.011628,0.000000,0.011628
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


In [62]:
num_top_venues = 5

for hood in DT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4  American Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2  Farmers Market  0.04
3        Beer Bar  0.04
4            Café  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4               Bar  0.07


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.07
2  Italian Restaurant  0.04
3              Bakery  0.04
4          Restaurant  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.06
3  Chin

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Thai Restaurant,Cosmetics Shop,Asian Restaurant,Bakery,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Steakhouse,Cheese Shop,Café,Hotel
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Bar
3,"Cabbagetown, St. James Town",Café,Coffee Shop,Restaurant,Italian Restaurant,Pub,Bakery,Market,Pizza Place,Park,Chinese Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Ice Cream Shop,Chinese Restaurant,Sandwich Place,Japanese Restaurant,Spa,Fried Chicken Joint
5,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Burger Joint
6,Christie,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Restaurant,Convenience Store,Coffee Shop
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gastropub,Pub,Men's Store,Bubble Tea Shop,Burger Joint
8,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Deli / Bodega,Bakery,Steakhouse,Gym
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bar,Gastropub,Deli / Bodega,American Restaurant,Japanese Restaurant


In [67]:
# run K-means to cluster the neighborhoods into 5 clusters
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 0, 2, 4, 0, 0, 0], dtype=int32)

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

DT_merge = DT_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merge = DT_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_merge.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Building,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Café,Coffee Shop,Restaurant,Italian Restaurant,Pub,Bakery,Market,Pizza Place,Park,Chinese Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gastropub,Pub,Men's Store,Bubble Tea Shop,Burger Joint
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Gym / Fitness Center,Café,Pub,Mexican Restaurant,Theater,Restaurant,Breakfast Spot
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Restaurant,Diner,Ice Cream Shop,Italian Restaurant,Japanese Restaurant


## Examine the clusters: 

In [69]:
DT_merge.loc[DT_merge['Cluster'] == 0, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Cabbagetown, St. James Town",0,Café,Coffee Shop,Restaurant,Italian Restaurant,Pub,Bakery,Market,Pizza Place,Park,Chinese Restaurant
2,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Gastropub,Pub,Men's Store,Bubble Tea Shop,Burger Joint
3,"Harbourfront, Regent Park",0,Coffee Shop,Park,Bakery,Gym / Fitness Center,Café,Pub,Mexican Restaurant,Theater,Restaurant,Breakfast Spot
4,"Ryerson, Garden District",0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Café,Restaurant,Diner,Ice Cream Shop,Italian Restaurant,Japanese Restaurant
5,St. James Town,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Cocktail Bar,Clothing Store,Bakery,Beer Bar,Cosmetics Shop
6,Berczy Park,0,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Steakhouse,Cheese Shop,Café,Hotel
7,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Ice Cream Shop,Chinese Restaurant,Sandwich Place,Japanese Restaurant,Spa,Fried Chicken Joint
8,"Adelaide, King, Richmond",0,Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Thai Restaurant,Cosmetics Shop,Asian Restaurant,Bakery,Hotel
9,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Brewery,Pizza Place,Scenic Lookout,Bakery,Sporting Goods Shop
10,"Design Exchange, Toronto Dominion Centre",0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bar,Gastropub,Deli / Bodega,American Restaurant,Japanese Restaurant


In [70]:
DT_merge.loc[DT_merge['Cluster'] == 1, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,1,Park,Playground,Trail,Building,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [71]:
DT_merge.loc[DT_merge['Cluster'] == 2, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Harbord, University of Toronto",2,Café,Restaurant,Japanese Restaurant,Bar,Bakery,Bookstore,Beer Bar,Beer Store,Dessert Shop,Comfort Food Restaurant
13,"Chinatown, Grange Park, Kensington Market",2,Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Burger Joint


In [72]:
DT_merge.loc[DT_merge['Cluster'] == 3, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]

,Neighborhood,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",3,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Bar
